## Mount with your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Setup darknet environment

In [ ]:
import os

# Download and compile darknet_for_colab
!git clone https://github.com/CZ3004-Group-13/darknet_for_colab.git
%cd darknet_for_colab
!make
!chmod +x ./darknet

## Copy yolov4 last trained weights

In [ ]:
!cp /content/drive/MyDrive/YOLOv4_weight/backup/yolov4_custom_train_final.weights /content/darknet_for_colab

## Download custom dataset (yolo format)

In [ ]:
%cd data

# Download data set from google drive
!cp /content/drive/MyDrive/ts.zip /content/darknet_for_colab/data
!unzip ts.zip
!rm -f ts.zip
!ls
%cd ..

## Modify yolov4 architecture

**Double click on file `yolov4_config.py` to modify the hyperpameters directly from Colab environment**

I will train my dataset with these parameters:
 - classes= 31, 
 - max_batches=1000
 - batch=64
 - subdivisions=16
 - width=256
 - height=256
 - ...

In [ ]:
!python yolov4_setup.py

## Create symbolic link in our Drive

Make sure that you laready created directory _YOLOv4_weight/backup_ in your Drive

In [ ]:
# Delete backup folder from our runtime
!rm /content/darknet_for_colab/backup -r

# Create Symlinks so we can save trained weight in our Google Drive
# Create folder YOLOv4_weight/backup in your Drive to store trained weights
!ln -s /content/drive/'My Drive'/YOLOv4_weight/backup /content/darknet_for_colab

## Training

In [ ]:
!./darknet detector train data/yolov4.data cfg/yolov4_custom_train.cfg yolov4_custom_train_final.weights -dont_show -map -clear
# If you get CUDA out of memory adjust subdivisions above,
# adjust max batches down for shorter training above